In [1]:
import xarray as xr

In [ ]:
historical_dir = r"/oscar/home/pvankatw/data/pvankatw/CMIP/access_cm2/historical"
historical = xr.open_mfdataset(f"{historical_dir}/*.nc", combine="by_coords", chunks=None)
historical.load()

<xarray.Dataset> Size: 657MB
Dimensions:    (time: 1980, bnds: 2, lat: 144, lon: 192)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * lat        (lat) float64 1kB -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
  * lon        (lon) float64 2kB 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 32kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 2kB dask.array<chunksize=(144, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 3kB dask.array<chunksize=(192, 2), meta=np.ndarray>
    evspsbl    (time, lat, lon) float32 219MB dask.array<chunksize=(1, 144, 192), meta=np.ndarray>
    pr         (time, lat, lon) float32 219MB dask.array<chunksize=(1, 144, 192), meta=np.ndarray>
    ts         (time, lat, lon) float32 219MB dask.array<chunksize=(1, 144, 192), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2019-11-08T08:37:21Z
    ...                     ...
    variable_id:            evspsbl
    variant_label:          r1i1p1f1
    version:                v20191108
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/fd5c749a-153a-4e0b-8156-609430899c87
    license:                CMIP6 model data produced by CSIRO is licensed un...

In [12]:
clim = xr.open_dataset(r"/oscar/home/pvankatw/data/pvankatw/CMIP/access_cm2/ssp126/atmosphere/climatology.nc")

In [14]:
clim.pr.values

array([[3.7409638e-05, 3.7373909e-05, 3.7321814e-05, ..., 4.4635985e-05,
        4.4701479e-05, 4.4717530e-05],
       [3.7407292e-05, 3.7354672e-05, 3.7302550e-05, ..., 4.4620487e-05,
        4.4685999e-05, 4.4753026e-05],
       [3.7387559e-05, 3.7334914e-05, 3.7282782e-05, ..., 4.4603465e-05,
        4.4668996e-05, 4.4736054e-05],
       ...,
       [3.1483487e-05, 3.1522919e-05, 3.1561671e-05, ..., 3.9923001e-05,
        3.9935006e-05, 3.9946921e-05],
       [3.1525247e-05, 3.1564679e-05, 3.1603431e-05, ..., 3.9936178e-05,
        3.9948180e-05, 3.9960079e-05],
       [3.1573109e-05, 3.1607109e-05, 3.1645868e-05, ..., 3.9949446e-05,
        3.9961444e-05, 3.9967250e-05]], dtype=float32)